In [1]:
import os

os.chdir("..")

In [2]:
import torch
from src.models import swin_v2

config_path = "checkpoints/swin-v2-msi/config.yaml"

/home/ubuntu/miniconda3/envs/hydro/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
model, transforms, config = swin_v2(config_path)

/home/ubuntu/miniconda3/envs/hydro/lib/python3.10/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


=> merge config from checkpoints/swin-v2-msi/config.yaml


INFO:root:>>>>>>>>>> Fine-tuned from checkpoints/swin-v2-msi/ckpt_epoch_799.pth ..........
INFO:root:Detect pre-trained model, remove [encoder.] prefix.
INFO:root:>>>>>>>>>> Remapping pre-trained keys for SWIN ..........
INFO:root:_IncompatibleKeys(missing_keys=['layers.0.blocks.0.attn.relative_coords_table', 'layers.0.blocks.0.attn.relative_position_index', 'layers.0.blocks.1.attn_mask', 'layers.0.blocks.1.attn.relative_coords_table', 'layers.0.blocks.1.attn.relative_position_index', 'layers.1.blocks.0.attn.relative_coords_table', 'layers.1.blocks.0.attn.relative_position_index', 'layers.1.blocks.1.attn_mask', 'layers.1.blocks.1.attn.relative_coords_table', 'layers.1.blocks.1.attn.relative_position_index', 'layers.2.blocks.0.attn.relative_coords_table', 'layers.2.blocks.0.attn.relative_position_index', 'layers.2.blocks.1.attn.relative_coords_table', 'layers.2.blocks.1.attn.relative_position_index', 'layers.2.blocks.2.attn.relative_coords_table', 'layers.2.blocks.2.attn.relative_positi

In [4]:
model = model.to("cuda")

In [7]:
# Extract image level embeddings
with torch.inference_mode():
    x = torch.randn(1, 12, 256, 256).to("cuda")
    x = transforms(x)
    embeddings = model.forward_features(x)
    print(embeddings.shape)

torch.Size([1, 1024])


In [8]:
# Extract intermediate feature embeddings
with torch.inference_mode():
    x = torch.randn(1, 12, 256, 256).to("cuda")
    x = transforms(x)
    features = model.get_intermediate_layers(x, n=(0, 1, 2, 3, 4), reshape=True)
    for i, f in enumerate(features):
        print(i, f.shape)

0 torch.Size([1, 128, 64, 64])
1 torch.Size([1, 256, 32, 32])
2 torch.Size([1, 512, 16, 16])
3 torch.Size([1, 1024, 8, 8])
4 torch.Size([1, 1024, 8, 8])
